In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import dlib

def shape_to_list(shape):
    coords = []
    for i in range(0, 68):
        coords.append((shape.part(i).x, shape.part(i).y))
    return coords

# Specify the base path
path = '/content/drive/MyDrive/Video_data/'
LIP_MARGIN = 0.3
RESIZE = (64, 64)

# Download the model file
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2

# Path to the model file
datFile = "/content/shape_predictor_68_face_landmarks.dat"

# Load the face detector and landmark detector
face_detector = dlib.get_frontal_face_detector()
landmark_detector = dlib.shape_predictor(datFile)

# Iterate through each folder (p1, p2, p3, etc.)
for folder_name in ['p1', 'p2', 'p3']:
    VIDEO_PATH = os.path.join(path, folder_name)  # Path to videos
    RESULT_PATH = os.path.join(path, folder_name.replace('p', 'r'))  # Path to results

    video_list = os.listdir(VIDEO_PATH)  # Read video list

    for vid_name in video_list:
        vid_path = os.path.join(VIDEO_PATH, vid_name)
        vid = cv2.VideoCapture(vid_path)  # Read video

        print(vid_name)

        frame_buffer = []               # A list to hold frame images
        frame_buffer_color = []         # A list to hold original frame images
        while True:
            success, frame = vid.read()  # Read frame
            if not success:
                break  # Break if no frame to read left
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_buffer.append(gray)
            frame_buffer_color.append(frame)

        landmark_buffer = []  # A list to hold face landmark information
        for (i, image) in enumerate(frame_buffer):  # Iterate on frame buffer
            face_rects = face_detector(image, 1)  # Detect face
            if len(face_rects) < 1:  # No face detected
                print("No face detected: ", vid_path)
                break
            rect = face_rects[0]  # Proper number of face
            landmark = landmark_detector(image, rect)  # Detect face landmarks
            landmark = shape_to_list(landmark)
            landmark_buffer.append(landmark)

        cropped_buffer = []
        for (i, landmark) in enumerate(landmark_buffer):
            lip_landmark = landmark[48:68]  # Landmark corresponding to lip
            lip_x = sorted(lip_landmark, key=lambda pointx: pointx[0])
            lip_y = sorted(lip_landmark, key=lambda pointy: pointy[1])
            x_add = int((-lip_x[0][0] + lip_x[-1][0]) * LIP_MARGIN)
            y_add = int((-lip_y[0][1] + lip_y[-1][1]) * LIP_MARGIN)
            crop_pos = (lip_x[0][0] - x_add, lip_x[-1][0] + x_add, lip_y[0][1] - y_add, lip_y[-1][1] + y_add)
            cropped = frame_buffer_color[i][crop_pos[2]:crop_pos[3], crop_pos[0]:crop_pos[1]]
            cropped = cv2.resize(cropped, (RESIZE[0], RESIZE[1]), interpolation=cv2.INTER_CUBIC)
            cropped_buffer.append(cropped)

        # Save results
        label = vid_name.replace(".mp4", "_")
        directory = RESULT_PATH + "/"
        if not os.path.exists(directory):
            os.makedirs(directory)

        for (i, image) in enumerate(cropped_buffer):
            cv2.imwrite(os.path.join(directory, "%s%d" % (label, (i + 1)) + ".jpg"), image)

        vid.release()
